In [1]:
import networkx as nx 
from matplotlib import pyplot as plt
import numpy as np
import networkx as nx
import torch
import copy
from gsnn.simulate.nx2pyg import nx2pyg
from gsnn.simulate.simulate import simulate
from gsnn.optim.Environment import Environment
from gsnn.optim.BayesOpt import BayesOptAgent
from gsnn.simulate.simulate import simulate_3_in_3_out
import argparse

# for reproducibility 
torch.manual_seed(0)
np.random.seed(0)

%load_ext autoreload
%autoreload 2

/home/teddy/miniconda3/envs/gsnn-lib/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/teddy/miniconda3/envs/gsnn-lib/lib/python3.12/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc
collect2: error: ld returned 1 exit status

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/usr/bin/ld: cannot find -lnvrtc
collect2: error: ld returned 1 exit status

OK


[WARNING 07-07 16:54:58] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


ImportError: cannot import name 'ParameterType' from 'ax' (/home/teddy/miniconda3/envs/gsnn-lib/lib/python3.12/site-packages/ax/__init__.py)

# Bayesian optimization for structure optimization 

In [2]:
G, pos, x_train, x_test, y_train, y_test, \
    input_nodes, function_nodes, output_nodes = simulate_3_in_3_out(n_train=50, n_test=250, noise_scale=0.05)

plt.figure(figsize=(8, 6))
nx.draw_networkx(G, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10, arrowstyle='->', arrowsize=20)
plt.title("Dummy Graph: 3 Inputs, 3 Outputs, 5 Function Nodes")
plt.show()

NameError: name 'simulate_3_in_3_out' is not defined

In [3]:
#special_functions = {'func1': lambda x: -np.mean(x), 'func2':lambda x: np.sum([np.exp(xx) for xx in x]), 
#                     'func0': lambda x: np.mean(([(xx-1)**2 for xx in x])), 'func3': lambda x: -np.mean(x) if all([xx > 0 for xx in x]) else np.mean(x)}
special_functions = None

x_train, x_test, y_train, y_test = simulate(G, n_train=100, n_test=100, input_nodes=input_nodes, output_nodes=output_nodes, 
                                            special_functions=special_functions, noise_scale=0.1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

x_train = torch.tensor(x_train, dtype=torch.float32).to(device)
x_test = torch.tensor(x_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

y_mu = y_train.mean(0); y_std = y_train.std(0)

y_train = (y_train - y_mu)/(y_std + 1e-8)
y_test = (y_test - y_mu)/(y_std + 1e-8)

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

NameError: name 'G' is not defined

In [4]:
data = nx2pyg(G, input_nodes, function_nodes, output_nodes) 

# add some false edges 
edge_index_dict = copy.deepcopy(data.edge_index_dict)

#edge_index_dict['function', 'to', 'function']  = torch.cat((edge_index_dict['function', 'to', 'function'], torch.tensor([[0, 1, 1, 2],
#                                                                                                                         [1, 0, 2, 1]], dtype=torch.long)), dim=-1)

edge_index_dict['input', 'to', 'function']  = torch.cat((edge_index_dict['input', 'to', 'function'], torch.tensor([[0, 0, 1, 1],
                                                                                                                   [1, 4, 0, 2]], dtype=torch.long)), dim=-1)

In [12]:
# GSNN model parameters
model_kwargs = {'edge_index_dict'                 : edge_index_dict, 
                    'node_names_dict'                 : data.node_names_dict,
                    'channels'                        : 20, 
                    'layers'                          : 5, 
                    'dropout'                         : 0,
                    'share_layers'                    : False,
                    'add_function_self_edges'         : True,
                    'norm'                            : 'layer'}

# GSNN training parameters
training_kwargs = {'lr':1e-3, 
                    'max_epochs':20, 
                    'batch':50,
                    'workers':5}

args = argparse.Namespace()
args.outer_iters = 100

# the action edge dict will be used to index the action for each edge 
# if and edge key is not provided, those values will not be optimized (always present)
# to specify a fixed edge within a given key, use the index -1 
action_edge_dict = {('input', 'to', 'function'): torch.arange(edge_index_dict['input', 'to', 'function'].shape[1])}
n_actions = sum([torch.unique(v).numel() for v in action_edge_dict.values()])
true_action = np.array([1.]*3 + [0.]*4)

env = Environment(action_edge_dict, train_dataset, test_dataset, model_kwargs, 
                 training_kwargs, metric='spearman', reward_type='auc', verbose=False,
                 raise_error_on_fail=True)

hoptim = BayesOptAgent(env, n_actions, action_labels=true_action, warmup=10, verbose=False, suppress_warnings=True)

for iter in range(args.outer_iters): 

    hoptim.step()


[INFO 11-13 13:58:37] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[ChoiceParameter(name='a0', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a1', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a2', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a3', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a4', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a5', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False), ChoiceParameter(name='a6', parameter_type=INT, values=[0, 1], is_ordered=True, sort_values=False)], parameter_constraints=[]).


[1, 1, 1, 1, 0, 0, 1] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 1 || best action acc: 0.714 || last reward: 7.969
[1, 1, 1, 1, 0, 0, 1] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 2 || best action acc: 0.714 || last reward: -1.073
[1, 1, 1, 1, 0, 0, 1] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 3 || best action acc: 0.714 || last reward: 2.663
[1, 0, 0, 0, 0, 0, 0] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 4 || best action acc: 0.714 || last reward: 8.428
[1, 1, 0, 0, 0, 0, 1] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 5 || best action acc: 0.714 || last reward: 8.725
[1, 1, 0, 0, 0, 0, 1] [1. 1. 1. 0. 0. 0. 0.]
			 --> iter: 6 || best action acc: 0.714 || last reward: 1.786


KeyboardInterrupt: 

In [6]:
hoptim.best_action

[1, 1, 1, 0, 0, 0, 0]

In [7]:
hoptim.client.get_trials_data_frame()

[WARNING 11-13 13:51:10] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


,trial_index,arm_name,trial_status,generation_method,reward,a0,a1,a2,a3,a4,a5,a6
0,0,0_0,COMPLETED,Sobol,8.959176,1,1,1,1,0,0,1
1,1,1_0,COMPLETED,Sobol,-1.327425,0,0,0,0,1,1,0
2,2,2_0,COMPLETED,Sobol,4.956872,0,1,1,1,1,1,1
3,3,3_0,COMPLETED,Sobol,8.356640,1,0,0,0,0,0,0
4,4,4_0,COMPLETED,Sobol,8.887157,1,1,0,0,0,0,1
5,5,5_0,COMPLETED,Sobol,5.383062,0,0,1,1,1,1,0
6,6,6_0,COMPLETED,Sobol,2.176450,0,1,0,0,1,1,1
7,7,7_0,COMPLETED,Sobol,8.234563,1,0,1,1,0,0,0
8,8,8_0,COMPLETED,Sobol,8.187505,1,1,0,1,1,0,1
9,9,9_0,COMPLETED,Sobol,5.689801,0,0,1,0,0,1,0
